In [6]:
import pandas as pd
import numpy as np
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [20]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
women=train_data.loc[train_data.Sex=="female"]["Survived"]
print(women.head())
rate_women=sum(women)/len(women) 
rate_women

1    1
2    1
3    1
8    1
9    1
Name: Survived, dtype: int64


0.7420382165605095

In [32]:
men=train_data.loc[train_data.Sex=="male"]["Survived"]
rate_male=sum(men)/len(men)
rate_male

0.18890814558058924

### 他のやり方で男女の生存率を求める

In [33]:
train_data.groupby(["Sex","Survived"]).size().reset_index(name="counts")

,Sex,Survived,counts
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


In [34]:
#男性の生存率
109/(468+109)

0.18890814558058924

In [35]:
y=train_data["Survived"]

#このままだとSexが文字列なので、数値に変換する
x=pd.get_dummies(train_data[["Pclass","Sex","SibSp","Parch"]])
X_test=pd.get_dummies(test_data[["Pclass","Sex","SibSp","Parch"]])

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(x,y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

---

In [39]:
from lightgbm import LGBMClassifier
model=LGBMClassifier(n_estimators=200,max_depth=10,random_state=1)
model.fit(X_train,y_train)
prediction=model.predict(X_test)

output=pd.DataFrame({"PassengerId":test_data.PassengerId,"Survived":prediction})
output.to_csv("Submission.csv",index=False) #index=Falseでindexを出力しないようにする


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000595 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

ValueError: array length 179 does not match index length 418